In [1]:
import ast

import polars as pl

df = pl.read_csv("../povarenok/povarenok_recipes_2021_06_16.csv").drop_nans()


df = df.with_columns(
    pl.col("ingredients")
    .map_elements(
        lambda x: list(ast.literal_eval(x).keys()), return_dtype=pl.List(pl.Utf8)
    )
    .alias("ingredients")
)
df = df.filter(
    pl.col("ingredients").is_not_null() & (pl.col("ingredients").list.len() > 0)
)


In [2]:
df_with_id = df.with_row_index(name="row_id")

# Разбиваем
df_exploded = df_with_id.explode("ingredients")

unique_names = df_exploded.select("name").unique().sort("name")
unique_names = unique_names.with_columns(pl.arange(0, pl.len()).alias("name_id"))

df_exploded = df_exploded.join(unique_names, on="name", how="left")


unique_ingredients = df_exploded.select("ingredients").unique().sort("ingredients")
unique_ingredients = unique_ingredients.with_columns(
    pl.arange(0, pl.len()).alias("ingredient_id")
)

# Присоединяем ID ингредиентов
df_joined = df_exploded.join(unique_ingredients, on="ingredients", how="left")

# Собираем обратно по row_id
df_restored = (
    df_joined.group_by("row_id", "name", "url", "name_id")
    .agg(
        [
            pl.col("ingredients").alias("ingredients"),
            pl.col("ingredient_id").alias("ingredient_ids"),
        ]
    )
    .sort("row_id")
    .drop("row_id")  # если не нужен
)

In [3]:
import numpy as np


def restored_train_test(df, test_size=0.4):
    rng = np.random.default_rng()
    is_train = pl.Series("is_train", (rng.random(df.height) > test_size).astype(bool))

    # Добавляем колонку к DataFrame
    df = df.with_columns(is_train)

    # Разделяем на train и test
    train = df.filter(pl.col("is_train") == True)
    test = df.filter(pl.col("is_train") == False)

    # Выводим размеры
    print(f"Train: {len(train)}, Test: {len(test)}")

    return train, test


train, val_test = restored_train_test(df_restored)
train

Train: 87758, Test: 58806


name,url,name_id,ingredients,ingredient_ids,is_train
str,str,i64,list[str],list[i64],bool
"""Рулетики""","""https://www.povarenok.ru/recip…",82487,"[""Сыр твердый"", ""Чеснок"", … ""Майонез""]","[898, 1050, … 414]",true
"""Салат ""Баклажанчик""""","""https://www.povarenok.ru/recip…",85053,"[""Баклажан"", ""Лук репчатый"", … ""Перец черный""]","[32, 406, … 631]",true
"""Куриные котлеты с картофельным…","""https://www.povarenok.ru/recip…",43969,"[""Фарш куриный"", ""Пюре картофельное"", … ""Масло растительное""]","[967, 698, … 450]",true
"""Рецепт вишневой наливки""","""https://www.povarenok.ru/recip…",79395,"[""Вишня"", ""Водка"", … ""Гвоздика""]","[112, 118, … 123]",true
"""Песочный пирог с тыквенным суф…","""https://www.povarenok.ru/recip…",66255,"[""Масло сливочное"", ""Сахар"", … ""Сахарная пудра""]","[451, 753, … 760]",true
…,…,…,…,…,…
"""Сухарник""","""https://www.povarenok.ru/recip…",104458,"[""Молоко"", ""Яйцо куриное"", ""Хлеб""]","[478, 1092, 1004]",true
"""Атеринопита""","""https://www.povarenok.ru/recip…",3495,"[""Рыба"", ""Помидор"", … ""Масло оливковое""]","[738, 667, … 446]",true
"""Массаман карри""","""https://www.povarenok.ru/recip…",52076,"[""Кардамон"", ""Фенхель"", … ""Орех мускатный""]","[260, 976, … 569]",true


In [4]:
val, test = restored_train_test(val_test)


Train: 35265, Test: 23541


In [5]:
(test.shape[0] + train.shape[0] + val.shape[0]) == df.shape[0]


True

In [6]:
test.shape[0] + train.shape[0] + val.shape[0]


146564

In [7]:
df.shape[0]


146564